In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac_cage as training_utils


from scipy import stats

2023-01-29 01:39:46.381057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 01:39:46.595988: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-29 01:39:46.596030: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-29 01:39:47.633928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-5')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-01-29 01:39:49.137987: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-29 01:39:49.138045: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-29 01:39:49.138076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-01-29 01:39:49.447444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 01:39:49.493678: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:58240
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-5


INFO:tensorflow:Initializing the TPU system: node-5


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
g = tf.random.Generator.from_seed(datetime.now().timestamp())
with strategy.scope():
    
    data_tr,data_val,data_val_TSS = training_utils.return_distributed_iterators("gs://picard-testing-176520/enformer_atac_rampage_basenji/preprocessed",
                                                                                "gs://picard-testing-176520/enformer_atac_rampage_paired_basenji_tss/preprocessed",
                                                                               GLOBAL_BATCH_SIZE,
                                                                               196608,
                                                                               10, 
                                                                               1536,
                                                                               320,
                                                                               128,
                                                                               8,
                                                                               50,
                                                                               strategy,
                                                                               options,
                                                                               g)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
with strategy.scope():
    inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/enformer_performer_FULL_atac_mean/models/enformer_performer_FULL_atac_mean_196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_K-relu_kernel_transformation_AD-0.05/iteration_20",
                                                              4)
    #inits = training_utils.get_initializers_enformer_conv("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                            dropout_rate = 0.05,
                            pointwise_dropout_rate=0.05,
                            input_length = 196608,
                            output_length = 1536,
                            final_output_length = 896,
                             num_heads = 4,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             hidden_size = 1552,
                             num_transformer_layers = 4,
                             d_model = 1552,
                             norm=True,
                             dim = 388, 
                            BN_momentum=0.90,
                             max_seq_length = 1536,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=True,
                             inits=inits,
                            freeze_conv_layers=True,
                            inits_type='enformer_performer')

True


In [5]:
with strategy.scope():
    scheduler1= tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1.0e-06,
        decay_steps=100000, alpha=1.0)
    scheduler1=WarmUp(initial_learning_rate=1.0e-06,
                                 warmup_steps=100,
                                 decay_schedule_fn=scheduler1)
    optimizer1 = tf.keras.optimizers.Adam(learning_rate=scheduler1,
                                          epsilon=1.0e-08)
    
    scheduler2= tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=5.0e-05,
    decay_steps=100000, alpha=1.0)
    scheduler2=WarmUp(initial_learning_rate=5.0e-05,
                                 warmup_steps=100,
                                 decay_schedule_fn=scheduler2)
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=scheduler2,
                                          epsilon=1.0e-08)
    
    
    scheduler3= tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=5.0e-04,
    decay_steps=100000, alpha=1.0)
    scheduler3=WarmUp(initial_learning_rate=5.0e-04,
                                 warmup_steps=100,
                                 decay_schedule_fn=scheduler3)
    optimizer3 = tf.keras.optimizers.Adam(learning_rate=scheduler3,
                                          epsilon=1.0e-08)
    
    optimizers_in=optimizer1,optimizer2,optimizer3
    

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step, val_step_TSS, build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                                             750,
                                                                                                             300,
                                                                                                             300,
                                                                                                              optimizers_in,
                                                                                                              strategy,
                                                                                                              metric_dict,
                                                                                                              GLOBAL_BATCH_SIZE,
                                                                                                                  1.0)


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(data_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))

        train_step(data_tr)

        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        
        val_step(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        print('hg_val_pearson: ' + str(metric_dict['PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_R2: ' + str(metric_dict['R2'].result()['R2'].numpy()))
        
        
        val_step_TSS(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map)
        
        cell_spec,gene_spec = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        

        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        

total params: tf.Tensor(81902641, shape=(), dtype=int32)
hg_train_loss: -0.013177392
hg_val_loss: 0.020918181
hg_val_pearson: [0.4167273]
hg_val_R2: [0.13373291]
cell spec corr: 0.635510793114661
gene_spec_corr:0.34080795434085503
completed epoch 1
duration(mins): 9.994677758216858
patience counter at: 0
hg_train_loss: -0.03999207
hg_val_loss: 0.0073499926
hg_val_pearson: [0.4171846]
hg_val_R2: [0.14728403]
hg_train_loss: -0.046407815
hg_val_loss: 0.010535456
hg_val_pearson: [0.40588716]
hg_val_R2: [0.16179371]
cell spec corr: 0.637723449949899
gene_spec_corr:0.26506400047514517
completed epoch 3
duration(mins): 9.42248212893804
patience counter at: 0
hg_train_loss: -0.043615483
hg_val_loss: -0.009590399
hg_val_pearson: [0.447315]
hg_val_R2: [0.19891745]
cell spec corr: 0.6314925159040732
gene_spec_corr:0.3361655929792983
completed epoch 4
duration(mins): 9.424247781435648
patience counter at: 0
hg_train_loss: -0.047541164
hg_val_loss: 0.04986248
hg_val_pearson: [0.4431965]
hg_val_R2: 

In [7]:
inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/enformer_performer_FULL_atac_mean/models/enformer_performer_FULL_atac_mean_196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_K-relu_kernel_transformation_AD-0.05/iteration_20",
                                                           4)
print(len(inits))

151


In [9]:



path = "gs://picard-testing-176520/enformer_performer_FULL_atac_mean/models/enformer_performer_FULL_atac_mean_196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_K-relu_kernel_transformation_AD-0.05/iteration_20"


inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(path))
reader = tf.train.load_checkpoint(path)

In [ ]:
test = next(data_val_TSS)

In [ ]:
test['cage']

In [19]:
model.layers[7].layers[0].trainable_variables

[TPUDistributedVariable:{
   0: <tf.Variable 'aformer/shared_transformer/transformer_layer/layer_normalization/gamma:0' shape=(1552,) dtype=float32, numpy=
 array([0.05186217, 0.04480721, 0.01227075, ..., 0.00143302, 0.00401236,
        0.00116567], dtype=float32)>,
   1: <tf.Variable 'aformer/shared_transformer/transformer_layer/layer_normalization/gamma/replica_1:0' shape=(1552,) dtype=float32, numpy=
 array([0.05186217, 0.04480721, 0.01227075, ..., 0.00143302, 0.00401236,
        0.00116567], dtype=float32)>,
   2: <tf.Variable 'aformer/shared_transformer/transformer_layer/layer_normalization/gamma/replica_2:0' shape=(1552,) dtype=float32, numpy=
 array([0.05186217, 0.04480721, 0.01227075, ..., 0.00143302, 0.00401236,
        0.00116567], dtype=float32)>,
   3: <tf.Variable 'aformer/shared_transformer/transformer_layer/layer_normalization/gamma/replica_3:0' shape=(1552,) dtype=float32, numpy=
 array([0.05186217, 0.04480721, 0.01227075, ..., 0.00143302, 0.00401236,
        0.00116567

In [14]:
inside_checkpoint

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/moving_mean/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/moving_variance/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [5, 768, 768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weights-0/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weig